In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from bracketology_rl import Bracket
from bracketgym import TournamentEnv
import warnings
import gym
import numpy as np
from tourney_tools import * 
warnings.filterwarnings('ignore')

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_30264/2451804577.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:

from stable_baselines3 import DQN,DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import ProgressBarCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import StopTrainingOnNoModelImprovement
from stable_baselines3.common.monitor import Monitor

In [3]:
import tensorboard

In [4]:
from pkg_resources import resource_filename
filepath = resource_filename('bracketology_rl', 'data/brackets_2022.json')
with open(filepath, 'r') as f:
    brackets_dict = json.load(f)

In [5]:
years = list(range(2003,2019))
year=np.random.choice(years)

bracket = Bracket(2022)

team_data=pd.read_csv('Process_data/pm_w_names.csv')


In [6]:
env=TournamentEnv(team_data=team_data,verbose=False,season=2022)


Reset! Sim 0 Bracket for year 2022:   0%|          | 0/120000 [00:00<?, ?it/s]

In [7]:
action=np.random.randn(env.action_space.shape[0])
env.step(action)


(array([ 1.        , -0.2519855 , -0.11848334, ...,  0.        ,
         0.        ,  0.        ], dtype=float32),
 15,
 False,
 {'round': 'first round', 'score': 15, 'num_correct': 15})

In [8]:
## tensor board_log
tensorboard_log = "Logs/"
# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
# callback=ProgressBarCallback()
model = DDPG("MlpPolicy", env, 
                        action_noise=action_noise,
                        gamma=.99,
                        learning_rate=.01,
                        gradient_steps=-1, 
                        verbose=0,
                        tensorboard_log=tensorboard_log,
                        )
# model.predict(observation)

In [9]:
obs=env.reset()
done=False
while not done:
    action=model.predict(obs)[0]

    obs,score, done,info=env.step(action)
    print(info)

Reset! Sim 1 Bracket for year 2014:   0%|          | 4/120000 [00:00<2:30:32, 13.28it/s]

{'round': 'first round', 'score': 44, 'num_correct': 22}
{'round': 'second round', 'score': 24, 'num_correct': 26}
{'round': 'sweet sixteen', 'score': 0, 'num_correct': 26}
{'round': 'elite eight', 'score': 0, 'num_correct': 26}
{'round': 'final four', 'score': 0, 'num_correct': 26}
{'round': 'championship', 'score': 0, 'num_correct': 26}


In [10]:
timesteps=50000000
try:
    model.learn(total_timesteps=timesteps, log_interval=10,reset_num_timesteps=True,tb_log_name='Tourney_bot',)
except KeyboardInterrupt:
    pass

Reset! Sim 2 Bracket for year 2014:   0%|          | 9/120000 [00:00<1:32:46, 21.55it/s]

In [ ]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), deterministic=False, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
273/6